<a href="https://colab.research.google.com/github/saverin0/llms_workshops_files/blob/main/oxford_llm_workshop_8_LLMs_alignment_with_RLHF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip setuptools wheel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [1]:
%%capture
!pip install "transformers==4.44.2" "datasets==2.21.0" "peft==0.12.0" "trl==0.10.1" "accelerate==1.0.1" "evaluate==0.4.2" "torch>=2.2" "tokenizers==0.19.1"

In [2]:
import torch
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

from trl import PPOConfig, PPOTrainer, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

from peft import LoraConfig, get_peft_model, TaskType

In [3]:
config = PPOConfig(
    model_name="lvwerra/gpt2-imdb",
    learning_rate=1.41e-5,
    mini_batch_size=4,        # small to start
    batch_size=16,            # fits on most GPUs
    gradient_accumulation_steps=1,
    ppo_epochs=4,
    seed=42,
    log_with=None,            # set to "wandb" if you use Weights & Biases
)

sent_kwargs = {
    "return_all_scores": True,
    "function_to_apply": "none",
    "batch_size": 16,
}

In [4]:
from trl.core import LengthSampler

def build_dataset(config, dataset_name="imdb", input_min_text_length=2, input_max_text_length=8):
    """
    Build dataset for training using Hugging Face datasets.
    It prepares short 'query' prompts from IMDB reviews.
    """
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    # ensure a pad token is set for GPT-2 family
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Load IMDB train split
    ds = load_dataset(dataset_name, split="train")

    # Standardize column name
    if "text" in ds.column_names:
        ds = ds.rename_columns({"text": "review"})
    elif "review" not in ds.column_names:
        raise ValueError(f"IMDB columns not as expected: {ds.column_names}")

    # Filter for longer reviews to allow continuation
    ds = ds.filter(lambda x: isinstance(x["review"], str) and len(x["review"]) > 200)

    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        # Take a short prefix from each review as the query
        ids = tokenizer.encode(sample["review"])
        ids = ids[: input_size()]
        sample["input_ids"] = ids
        sample["query"] = tokenizer.decode(ids)
        return sample

    ds = ds.map(tokenize)
    ds.set_format(type="torch")
    return ds

dataset = build_dataset(config)

def collator(data):
    return {k: [d[k] for d in data] for k in data[0]}

tokenizer_config.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/24895 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1168 > 1024). Running this sequence through the model will result in indexing errors


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import AutoModelForCausalLMWithValueHead

# Load base policy model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base_policy = AutoModelForCausalLM.from_pretrained(
    config.model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
)

# Apply LoRA to the policy
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["c_attn", "c_proj"],  # typical for GPT-2
)
policy_lora = get_peft_model(base_policy, peft_config)
policy_lora.print_trainable_parameters()

# Add value head for PPO
model = AutoModelForCausalLMWithValueHead.from_pretrained(
    policy_lora,
    torch_dtype=policy_lora.dtype,
)

# Reference model (no LoRA, no value head training)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(
    config.model_name,
    torch_dtype=policy_lora.dtype,
)

# For generation padding
tokenizer.pad_token = tokenizer.eos_token

pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:1091: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475


In [6]:
ppo_trainer = PPOTrainer(
    config=config,
    model=model,
    ref_model=ref_model,
    tokenizer=tokenizer,
    dataset=dataset,
    data_collator=collator,
)

print(f"PPO Trainer initialized. Device: {ppo_trainer.accelerator.device}")

PPO Trainer initialized. Device: cuda


In [7]:
from transformers import pipeline

# Use the PPO trainer's device; special-case single-process for HF pipeline
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"

sentiment_pipe = pipeline(
    task="sentiment-analysis",
    model="lvwerra/distilbert-imdb",
    device=device,
)

sent_kwargs = {
    "return_all_scores": True,
    "function_to_apply": "none",
    "batch_size": 16,
}

# Quick sanity check
print("Neg sample:", sentiment_pipe("this movie was really bad!!", **sent_kwargs))
print("Pos sample:", sentiment_pipe("this movie was really good!!", **sent_kwargs))

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Neg sample: [[{'label': 'NEGATIVE', 'score': 2.3350484371185303}, {'label': 'POSITIVE', 'score': -2.726576328277588}]]
Pos sample: [[{'label': 'NEGATIVE', 'score': -2.294790267944336}, {'label': 'POSITIVE', 'score': 2.557040214538574}]]


In [11]:
from trl.core import LengthSampler
from tqdm import tqdm

# Generation configs
gen_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}

# Response length sampler
output_min_length = 4
output_max_length = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)

generation_kwargs = dict(gen_kwargs)  # copy

num_steps = 10  # keep small for a quick run; increase later

# Helper to safely convert stats to printable format
def to_py(x):
    try:
        if hasattr(x, "item"):
            return x.item()
        if isinstance(x, (list, tuple)):
            return [to_py(y) for y in x]
        if hasattr(x, "tolist"):
            t = x.tolist()
            if isinstance(t, (int, float)):
                return t
            return t
        return x
    except Exception:
        return str(x)

for step, batch in tqdm(enumerate(ppo_trainer.dataloader), total=num_steps):
    if step >= num_steps:
        break

    query_tensors = batch["input_ids"]

    # 1) Generate responses with current policy
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])

    # Also keep decoded strings for reward model input
    batch["response"] = [tokenizer.decode(r) for r in response_tensors]

    # 2) Compute sentiment rewards using the reward model
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)

    # Extract positive sentiment scores (pipe_outputs is list of lists of dicts)
    rewards = []
    for outs in pipe_outputs:
        # Find the POSITIVE label score
        pos = next(o for o in outs if o["label"].upper().startswith("POS"))
        rewards.append(torch.tensor(pos["score"]))

    # 3) PPO optimization step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)

    # Optional: print a few stats safely
    short = {}
    for k, v in list(stats.items()):
        if len(short) >= 8:
            break
        short[k] = to_py(v)

    print(f"Step {step}: {short}")

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
 10%|█         | 1/10 [00:03<00:35,  3.92s/it]

Step 0: {'objective/kl': 0.006962994579225779, 'objective/kl_dist': '[ 0.06078577 -0.01874999 -0.05354285  0.02074313  0.07031024  0.01289365\n -0.00742745 -0.02442813 -0.11948596 -0.02700925  0.01695353 -0.04024032\n -0.00184906 -0.01716274  0.12055111  0.11906624]', 'objective/logprobs': '[[-3.29031205e+00 -2.05686474e+00 -6.60799408e+00 -3.09875298e+00\n  -5.83467007e+00 -2.22267604e+00 -2.23946714e+00 -2.25662780e+00\n  -4.46732235e+00 -4.84113550e+00 -4.69295454e+00 -2.00723624e+00\n  -1.06328144e+01 -1.54689908e+00 -1.75061727e+00 -9.71703339e+00\n  -3.81040359e+00 -3.86117792e+00 -3.95989799e+00 -4.01697922e+00]\n [-8.26208687e+00 -7.18007708e+00 -7.76755288e-02 -6.75038099e-01\n  -1.17049015e+00 -1.02320824e+01 -5.05077314e+00 -7.85085917e-01\n  -1.72748637e+00 -2.07094574e+00 -6.47831726e+00 -4.51670647e+00\n  -1.22854996e+01 -2.69351912e+00 -2.79186201e+00 -2.85891008e+00\n  -3.02484035e+00 -3.08969545e+00 -3.17026520e+00 -3.20050669e+00]\n [-5.47051144e+00 -1.81663370e+00 -6

 20%|██        | 2/10 [00:07<00:30,  3.79s/it]

Step 1: {'objective/kl': -0.01041182316839695, 'objective/kl_dist': '[-0.04587734  0.00287406 -0.04769492  0.02852726 -0.00921872 -0.01941022\n -0.12188326 -0.05038357  0.0692209   0.02324492  0.01246226 -0.0082702\n  0.04218006 -0.01667064 -0.00640014 -0.01928961]', 'objective/logprobs': '[[-5.3839731e+00 -8.8991823e+00 -9.7419214e-01 -1.2824804e+01\n  -3.1838827e+00 -1.0464891e+01 -3.1037655e+00 -4.9572241e-01\n  -2.1387944e+00 -4.4350853e+00 -1.2234937e+00 -6.1251144e+00\n  -7.5464544e+00 -7.1701555e+00 -7.4273663e+00 -1.8837144e+00\n  -1.2282405e+00 -3.3579555e+00 -2.8143685e+00]\n [-8.0618773e+00 -1.9367571e+00 -1.8174028e+00 -7.8073602e+00\n  -5.9014597e+00 -3.9704032e+00 -8.8371724e-01 -8.7018833e+00\n  -1.6609836e-01 -1.7565603e+00 -2.1692381e+00 -4.2370048e+00\n  -6.6713924e+00 -5.1436548e+00 -2.1942437e-01 -3.2385900e+00\n  -1.0322816e+01 -9.3841934e+00 -1.6955931e+00]\n [-9.9085083e+00 -2.1807499e+00 -4.1340947e+00 -2.2031696e+00\n  -2.0475092e+00 -7.5474992e+00 -4.4161344e+

 30%|███       | 3/10 [00:10<00:24,  3.54s/it]

...

Step 7: {'objective/kl': 0.027147747576236725, 'objective/kl_dist': '[ 0.04399065 -0.05890161 -0.07207608  0.2461732  -0.10279405 -0.00832224\n  0.35776767 -0.0764302  -0.0093945  -0.2055974  -0.00310428  0.11304031\n -0.07787061  0.17298841  0.10435873  0.01053596]', 'objective/logprobs': '[[-1.87166119e+00 -8.22038841e+00 -3.79365420e+00 -5.30211115e+00\n  -1.51490555e+01 -1.29121971e+00 -1.03498220e+01 -8.54270363e+00\n  -2.67081596e-02 -6.82489872e+00 -8.18949413e+00 -8.90173244e+00\n  -9.32022953e+00 -7.15937138e+00 -1.38572061e+00 -6.03166246e+00\n  -1.28212690e+01 -3.48505092e+00 -3.59363675e+00 -3.60807729e+00]\n [-5.54784918e+00 -3.15050316e+00 -3.36786914e+00 -3.23554516e+00\n  -2.51742506e+00 -2.58506441e+00 -1.75235963e+00 -7.02020526e-01\n  -4.36291599e+00 -5.64616108e+00 -5.09695768e+00 -1.12808352e+01\n  -1.88668156e+00 -1.99584889e+00 -2.12707853e+00 -2.22421288e+00\n  -2.43243146e+00 -2.55908418e+00 -2.68708396e+00 -2.76818705e+00]\n [-8.82844734e+00 -6.40366852e-01 -1

 90%|█████████ | 9/10 [00:32<00:03,  3.74s/it]

Step 8: {'objective/kl': 0.06409966945648193, 'objective/kl_dist': '[-0.22563207 -0.18798415  0.1836124   0.14921099  0.1518544   0.31150913\n -0.12313414  0.10715523 -0.05673565  0.22108507  0.05570662  0.15389705\n  0.2262572   0.17826818  0.08262283 -0.20209837]', 'objective/logprobs': '[[-3.95339465e+00 -4.95814419e+00 -2.34637499e+00 -1.10321236e+01\n  -3.49469233e+00 -2.39853287e+00 -5.47046614e+00 -2.13360190e+00\n  -3.40036392e+00 -2.30960751e+00 -5.32180786e+00 -3.60375667e+00\n  -2.22011089e+00 -9.71106434e+00 -8.80098462e-01 -9.21758938e+00\n  -6.72246456e-01 -1.17561455e+01 -2.74310994e+00]\n [-7.37992382e+00 -1.25730348e+00 -2.04326248e+00 -5.72960949e+00\n  -1.43554652e+00 -3.00756145e+00 -5.89634562e+00 -1.85947227e+00\n  -9.00640106e+00 -1.17030449e+01 -1.87056497e-01 -5.96456718e+00\n  -1.19663119e+00 -7.36467409e+00 -7.60490847e+00 -4.64249420e+00\n  -5.47820807e+00 -1.03121405e+01 -1.12067471e+01]\n [-1.05757904e+01 -4.86122513e+00 -4.49799538e+00 -3.81310439e+00\n  

100%|██████████| 10/10 [00:35<00:00,  3.58s/it]

Step 9: {'objective/kl': 0.056333277374506, 'objective/kl_dist': '[-0.12797081 -0.190736   -0.02201978  0.09332725  0.04889774  0.123285\n  0.09028721 -0.03886089  0.11446917  0.11740624  0.02295828  0.18012452\n -0.09400129  0.4536546   0.12783241  0.0026788 ]', 'objective/logprobs': '[[-3.96099353e+00 -3.27805185e+00 -5.19003057e+00 -2.76181791e-02\n  -2.47307038e+00 -2.40311861e+00 -4.47958136e+00 -4.76386309e+00\n  -4.49063158e+00 -1.40354192e+00 -4.13169670e+00 -7.52153778e+00\n  -2.17741203e+00 -4.28768635e+00 -4.15949297e+00 -4.17519093e+00\n  -4.23920155e+00 -4.22616911e+00 -4.25990200e+00 -4.24963951e+00]\n [-3.04190469e+00 -2.01471615e+00 -3.24544644e+00 -8.34783173e+00\n  -7.79434621e-01 -3.26180863e+00 -3.10689187e+00 -8.18385506e+00\n  -3.24331427e+00 -1.33175659e+00 -7.66240072e+00 -1.95357144e+00\n  -8.05221367e+00 -3.45714617e+00 -3.58280730e+00 -3.64174390e+00\n  -3.76563072e+00 -3.83137536e+00 -3.91441989e+00 -3.92276359e+00]\n [-2.44220519e+00 -2.32320571e+00 -4.3309

In [12]:
# Compare ref_model vs. trained model on a small random batch

bs = 8
game_data = {}

# Switch dataset to pandas to easily sample
dataset.set_format("pandas")
df_batch = dataset[:].sample(bs, random_state=42)
game_data["query"] = df_batch["query"].tolist()
query_tensors = df_batch["input_ids"].tolist()

response_tensors_ref, response_tensors = [], []

# Generate from reference and current policy
for i in range(bs):
    gen_len = output_length_sampler()
    q = torch.tensor(query_tensors[i]).unsqueeze(0).to(ppo_trainer.accelerator.device)

    out_ref = ref_model.generate(
        input_ids=q,
        max_new_tokens=gen_len,
        **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(out_ref)

    out_pol = model.generate(
        input_ids=q,
        max_new_tokens=gen_len,
        **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(out_pol)

# Decode
game_data["response (before)"] = [tokenizer.decode(t) for t in response_tensors_ref]
game_data["response (after)"] = [tokenizer.decode(t) for t in response_tensors]

# Sentiment rewards using the same sentiment_pipe and sent_kwargs (top_k=None)
texts_before = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
texts_after  = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]

def get_pos_scores(texts):
    outs = sentiment_pipe(texts, **sent_kwargs)
    scores = []
    for olist in outs:
        pos = next(o for o in olist if o["label"].upper().startswith("POS"))
        scores.append(pos["score"])
    return scores

game_data["rewards (before)"] = get_pos_scores(texts_before)
game_data["rewards (after)"]  = get_pos_scores(texts_after)

# Show as DataFrame
df_results = pd.DataFrame(game_data)
print(df_results[["query", "response (before)", "response (after)", "rewards (before)", "rewards (after)"]])

print("\nMean rewards:")
print(df_results[["rewards (before)", "rewards (after)"]].mean())

print("\nMedian rewards:")
print(df_results[["rewards (before)", "rewards (after)"]].median())

                                  query  \
0                             This film   
1                 "A death at a college   
2  Tim (Gary Daniels) wants desperately   
3              This movie was so great!   
4              This movie came and went   
5                          This film is   
6                             Tea Leoni   
7                Man on Fire was hot. I   

                                   response (before)  \
0          focuses almost entirely around Arbor Man,   
1  ," perhaps the 30 best references for horror m...   
2                      to leave Vienna, but Flich is   
3                         was so great!<|endoftext|>   
4         , he collected his crew and all other crew   
5          suitable for people who just haven't seen   
6                                 (Rose Tyler). They   
7  'm writing this under the pressure of not agre...   

                                    response (after)  rewards (before)  \
0                        has a bi